In [23]:
%load_ext autoreload
%autoreload 2
%aimport algorithm_utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from algorithm_utils import *
import general_utils as gu

In [40]:
testing_length = 500
start_index = -8*testing_length - 1
pair_name = "XRPUSDT"

original_pair_df: pd.DataFrame = gu.load_local_data(pair_name)

if testing_length > 0:
    pair_df: pd.DataFrame = original_pair_df.iloc[start_index:start_index + testing_length].reset_index(drop=True)
else:
    pair_df: pd.DataFrame = original_pair_df

# Higher timeframe data for determining starting point
htf_pair_df: pd.DataFrame = gu.load_higher_tf_data(pair_name=pair_name)

print("LTF length is", len(pair_df))
print("HTF length is", len(htf_pair_df))



LTF length is 500
HTF length is 5845


In [46]:
gu.reset_logs()

print("Original pair_df starting point is at time", pair_df.iloc[0].time)

algo = Algo(pair_df, pair_name, allowed_verbosity=1)
initial_data_start_time = algo.pair_df.iloc[0].time

corrected_pair_df = create_filtered_pair_df_with_corrected_starting_point(htf_pair_df, initial_data_start_time,
                                                                          original_pair_df)

print("Updated pair_df starting point to", corrected_pair_df.iloc[0].time)

algo = Algo(corrected_pair_df, "BTCUSDT", allowed_verbosity=0)

algo.init_zigzag(last_pivot_type="valley", last_pivot_candle_pdi=0)
h_o_starting_point: int = algo.zigzag_df.iloc[0].pdi

algo.calc_h_o_zigzag(h_o_starting_point)


# algo.h_o_indices.append(87)
# algo.h_o_indices.append(315)
# algo.h_o_indices.append(431)
# print(algo.find_lpls())
# h_o_zigzag_df = generate_h_o_zigzag(zigzag_df)
# pbos_df = h_o_zigzag_df.loc[h_o_zigzag_df.is_pbos]
# bos_df = find_confirmed_boss(pbos_df, pair_df)
# lpl_df = find_lpls(bos_df, zigzag_df)
# lplb_df = find_lplbs(bos_df, zigzag_df)

# boxes_list = []
# for lpl in lpl_df.itertuples():
#     start_index = lpl.pair_df_index
#     base_candle = Candle.create(pair_df.iloc[lpl.pair_df_index])
#     box_type = "long" if lpl.pivot_type == "valley" else "short"
#     box_to_add = Box(base_candle, b|ox_type)
#     box_to_add.check_box_ent

#     ries(pair_df)
#     boxes_list.append(box_to_add)


Original pair_df starting point is at time 2024-07-21 08:00:00
Updated pair_df starting point to 2024-07-19 10:00:00


In [47]:
import constants

pt = gu.PlottingTool(width=1200, height=500, x_axis_type="time")
pt.draw_candlesticks(corrected_pair_df, label="LTF")

truncated_htf_pair_df = htf_pair_df.iloc[-constants.starting_point_backtrack_window:].reset_index()
pt.draw_candlesticks(truncated_htf_pair_df, label="HTF")
htf_algo = Algo(truncated_htf_pair_df, "HTF Data", allowed_verbosity=0)
htf_algo.init_zigzag()
first_zigzag_pivot_pdi: int = htf_algo.zigzag_df.iloc[0].pdi
htf_algo.calc_h_o_zigzag(first_zigzag_pivot_pdi)

pt.draw_points_with_label(algo.zigzag_df[algo.zigzag_df.pdi.isin(algo.h_o_indices)].time,
                          algo.zigzag_df[algo.zigzag_df.pdi.isin(algo.h_o_indices)].pivot_value,
                          draw_line=True)

pt.draw_points_with_label(htf_algo.zigzag_df[htf_algo.zigzag_df.pdi.isin(htf_algo.h_o_indices)].time,
                          htf_algo.zigzag_df[htf_algo.zigzag_df.pdi.isin(htf_algo.h_o_indices)].pivot_value,
                          draw_line=True, color="orange")
pt.show()
pt.save_plot(scale=4)

In [30]:
pt = gu.PlottingTool(width=1400, height=500)
pt.draw_candlesticks(corrected_pair_df)
pt.draw_zigzag(algo.zigzag_df)

# pt.draw_zigzag(h_o_zigzag_df, title='Higher order Zigzag', color='black')
# plot_type = "descending" if algo.zigzag_df[algo.zigzag_df.pdi == algo.starting_pdi].iloc[0].pivot_type == "peak" else "ascending"
plot_type = "ascending"
# if algo.zigzag_df[algo.zigzag_df.pdi == algo.starting_pdi].iloc[0].pivot_type == "valley":
#     pt.draw_points_with_label(algo.pbos_indices, 
#                               algo.pair_df.iloc[algo.pbos_indices].high, 
#                               "PBOS", color="purple")
#     pt.draw_points_with_label(algo.valley_lpls.pdi, algo.valley_lpls.pivot_value, "Broken LPL", color="yellow")
#     pt.draw_points_with_label(algo.valley_broken_lpls.pdi, algo.valley_broken_lpls.pivot_value, "Broken LPL", color="yellow")
# else:
#     pt.draw_points_with_label(algo.pbos_indices, 
#                               algo.pair_df.iloc[algo.pbos_indices].low, 
#                               "PBOS", color="purple")
#     pt.draw_points_with_label(algo.peak_lpls.pdi, algo.peak_lpls.pivot_value, "LPL", color="red")
#     pt.draw_points_with_label(algo.peak_broken_lpls.pdi, algo.peak_broken_lpls.pivot_value, "Broken LPL", color="yellow")
invert = False

if invert:
    plot_type = "ascending" if plot_type == "descending" else "descending"

# if plot_type == "ascending":
# pt.draw_points_with_label(algo.pbos_indices,
#                           algo.pair_df.iloc[algo.pbos_indices].high,
#                           "PBOS", color="purple")
# pt.draw_points_with_label(algo.valley_lpls.pdi, algo.valley_lpls.pivot_value, "LPL", color="red")
# pt.draw_points_with_label(algo.valley_broken_lpls.pdi, algo.valley_broken_lpls.pivot_value, "Valley broken LPL", color="orange")
# else:
# pt.draw_points_with_label(algo.pbos_indices,
#                           algo.pair_df.iloc[algo.pbos_indices].low,
#                           "PBOS", color="purple")
# pt.draw_points_with_label(algo.peak_lpls.pdi, algo.peak_lpls.pivot_value, "LPL", color="red")
# pt.draw_points_with_label(algo.peak_broken_lpls.pdi, algo.peak_broken_lpls.pivot_value, "Peak broken LPL", color="orange")

pt.draw_points_with_label(algo.zigzag_df[algo.zigzag_df.pdi.isin(algo.h_o_indices)].pdi,
                          algo.zigzag_df[algo.zigzag_df.pdi.isin(algo.h_o_indices)].pivot_value,
                          draw_line=True)

# pt.draw_points_with_label(algo.peak_lpls.pair_df_index, algo.peak_lpls.pivot_value, "Peak LPL", color="green")
# pt.draw_points_with_label(bos_df.loc[bos_df.pivot_type == "valley"], "BOS-S", color="red")
# pt.draw_points_with_label(lplb_df.loc[lplb_df.pivot_type == "peak"], "LPLB-S", color="orange")
# pt.draw_points_with_label(lpl_df.loc[lpl_df.pivot_type == "peak"], "LPL-S", color="green")
# for box in boxes_list:
#     print(box.id)
#     pt.draw_box(box, pair_df_end_index=pair_df.last_valid_index())
# pt.draw_highlight()
# pt.draw_points_with_label(bos_df.loc[bos_df.pivot_type == "valley"], "BOS-S", color="blue")
pt.show()
pt.save_plot(scale=4)

In [164]:
algo.h_o_indices

[16, np.int64(57), 61, np.int64(80), 81, np.int64(125)]

In [11]:
algo.identify_fvgs()
pt = gu.PlottingTool(width=1200, height=500)
pt.draw_candlesticks(pair_df)
for fvg in algo.fvg_list:
    pt.fig.add_shape(type="rect",
                     x0=fvg.middle_candle - 1,
                     y0=fvg.fvg_lower,
                     x1=fvg.middle_candle + 1,
                     y1=fvg.fvg_upper,
                     fillcolor="blue",
                     opacity=0.5,
                     name="FVG")

pt.show()
pt.save_plot(scale=3)

ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido


In [133]:
htf_pair_df: pd.DataFrame = gu.load_higher_tf_data()
ltf_start_time: pd.Timestamp = algo.pair_df.iloc[0].time
find_starting_point(htf_pair_df, ltf_start_time)

2024-08-05 04:00:00
